# `BlockScheme` to represent time integration methods

Time-stepping methods are represented by objects inheriting from a base class `BlockScheme` defined in the `blockops.schemes` subpackage.
This class implements a `getBlockOperators(lamDt, phiName, chiName)` method that returns two `BlockOperator` objects that represent the $\phi$ and $\chi$ block operators for one block, such that:

$$
\phi {\bf u}_{n+1} = \chi {\bf u}_{n}.
$$

The `getBlockOperators` method uses two abstract method that should be overwritten in the children classes :

- `getBlockMatrices(lamDt)` : generate the block matrices for $\phi$ and $\chi$
- `getBlockCosts()` : estimate costs for $\phi$ and $\chi$

All available time integration methods are registered in the `SCHEMES` dictionnary of the `blockops.schemes` subpackage. This dictionnary contains all available classes that inherit from `BlockScheme` to implement one given type of time-stepping method.

In [1]:
from blockops.schemes import SCHEMES
SCHEMES

{'Collocation': blockops.schemes.coll.Collocation,
 'RungeKutta': blockops.schemes.rk.RungeKutta}

Here we can see that two time-stepping schemes are available, namely `Collocation` and `RungeKutta`. Those can be instantiated with a constructor having specific parameters, some inherited from the `BlockScheme` class, some specific to the time-stepping method.

## `Parameter` subclass to describe parameters

Looking closely to the `RungeKutta` class, we can access all its parameters through its `PARAMS` attribute:

In [2]:
rk = SCHEMES['RungeKutta']

rk.PARAMS

{'nPoints': Integer(default=None),
 'ptsType': MultipleChoices(default=EQUID),
 'quadType': MultipleChoices(default=LOBATTO),
 'form': MultipleChoices(default=Z2N),
 'rkScheme': MultipleChoices(default=BE),
 'nStepsPerPoint': Integer(default=1)}

Each parameter is represented by a specific `Parameter` class (defined in `blockops.utils.params` submodule), which stores:

- the documentation for this parameter
- what kind of value is expected
- what are the current default for the class (`default=None` indicate non optional parameters)

Concerning the `RungeKutta` scheme class, it requires several parameter, with only one (`nPoints`) that is not optional (_i.e_ don't have a pre-set default value).

Two types of parameters are used, namely the `PositiveNumber` and `MultipleChoices` parameter class. In order to see what type of value are expected by those parameters,
one can simply look at the docstring of those parameters classes:

In [3]:
rk.PARAMS['nPoints'].__doc__

'Parameter that accepts integer, eventually strictly positive'

In [4]:
rk.PARAMS['ptsType'].__doc__

'Parameter that accepts different parameter values or parameter types'


More information are also stored on the parameter itself. 
For instance, if we look at the `ptsType` parameter, we can look at its documentation looking at the `docs` attribute of the parameter class:

In [5]:
ptsType = rk.PARAMS['ptsType']
print(ptsType.docs)

Either the type of points (EQUID, LEGENDRE), or a list of given time
points in [0, 1].


The default value of this parameter for this class is then stored in the `default` attribute of the parameter class (`None` if parameter is not optional):

In [6]:
ptsType.default

'EQUID'

Each parameter class is implemented in the `blockops.utils.params` submodule, where one can see its particular implementation and eventual attributes.

For instance, the `MultipleChoices` parameter class allows different values or parameter type, and those are stored in the `choices` and `pTypes` attributes:

In [7]:
ptsType.choices, ptsType.pTypes

(['EQUID', 'LEGENDRE', 'CHEBY-1', 'CHEBY-2', 'CHEBY-3', 'CHEBY-4'],
 [CustomPoints(default=None)])

Here we can see that the user can choose between `EQUID`, `LEGENDRE`, ... or eventually put a a value accepted by a `CustomPoints` parameter type:

In [8]:
ptsType.pTypes[0].__doc__

'Parameter that accepts an ordered list of float values in [0, 1]'

## Accessing defaults and docs for all parameters

As described before, each `BlockScheme` subclass stores its parameters in the `PARAMS` attribute, from which documentation and default values can be retrieved individually for each parameter.
But additionally, those `BlockScheme` subclass also inherit from utility class methods allowing to retrieve defaults and docs for all parameters.

For instance, looking at the `RungeKutta` class:

In [9]:
SCHEMES['RungeKutta'].getParamsDefault()

{'nPoints': None,
 'ptsType': 'EQUID',
 'quadType': 'LOBATTO',
 'form': 'Z2N',
 'rkScheme': 'BE',
 'nStepsPerPoint': 1}

In [10]:
for param, doc in SCHEMES['RungeKutta'].getParamsDocs().items():
    print(f'--- {param} ---')
    print(doc)

--- nPoints ---
Number of time points in the block. Ignored if a custom list of points
is given for `ptsType`.
--- ptsType ---
Either the type of points (EQUID, LEGENDRE), or a list of given time
points in [0, 1].
--- quadType ---
Quadrature type used for the points in [0, 1]:
- LOBATTO -> 0 and 1 are included
- GAUSS -> neither 0 nor 1 are included
- RADAU-RIGHT -> only 1 is included
- RADAU-LEFT -> only 0 is included
--- form ---
Used formulation, either N2N (node-to-node) or Z2N (zero-to-node).
--- rkScheme ---
Name of the Runge-Kutta scheme (BE, FE, TRAP, RK4, ...).
--- nStepsPerPoint ---
Number of time-steps per block time point.


Or for the `Collocation` class:

In [11]:
SCHEMES['Collocation'].getParamsDefault()

{'nPoints': None,
 'ptsType': 'LEGENDRE',
 'quadType': 'LOBATTO',
 'form': 'Z2N',
 'collUpdate': False}

In [12]:
for param, doc in SCHEMES['Collocation'].getParamsDocs().items():
    print(f'--- {param} ---')
    print(doc)

--- nPoints ---
Number of time points in the block. Ignored if a custom list of points
is given for `ptsType`.
--- ptsType ---
Either the type of points (EQUID, LEGENDRE), or a list of given time
points in [0, 1].
--- quadType ---
Quadrature type used for the points in [0, 1]:
- LOBATTO -> 0 and 1 are included
- GAUSS -> neither 0 nor 1 are included
- RADAU-RIGHT -> only 1 is included
- RADAU-LEFT -> only 0 is included
--- form ---
Used formulation, either N2N (node-to-node) or Z2N (zero-to-node).
--- collUpdate ---
Wether to use or not the collocation update at the end of the step.


## Instantiation and use of a `BlockScheme` subclass

Once the user is set on a type of time-stepping method, the next step is to instantiate the corresponding `BlockScheme` subclass and use its `getBlockOperators(...)` method to generate the corresponding `BlockOperator` for one given value of $\lambda\Delta{T}$.

For instance, let us choose the `RungeKutta` time-stepping type, and look at the different parameters that have to be set:

In [13]:
BlockSchemeClass = SCHEMES['RungeKutta']
BlockSchemeClass.PARAMS

{'nPoints': Integer(default=None),
 'ptsType': MultipleChoices(default=EQUID),
 'quadType': MultipleChoices(default=LOBATTO),
 'form': MultipleChoices(default=Z2N),
 'rkScheme': MultipleChoices(default=BE),
 'nStepsPerPoint': Integer(default=1)}

Here only the number of points `nPoints` is non optional, which means that we can instantiate the `BlockScheme` subclass with 5 time points, for instance:

In [14]:
rkScheme = BlockSchemeClass(5)
rkScheme.PARAMS

{'nPoints': Integer(value=5),
 'ptsType': MultipleChoices(value=EQUID),
 'quadType': MultipleChoices(value=LOBATTO),
 'form': MultipleChoices(value=Z2N),
 'rkScheme': MultipleChoices(value=BE),
 'nStepsPerPoint': Integer(value=1)}

We see now that the `PARAMS` displays parameters with their type and value (no default anymore), since the class has been instantiated and parameters values have been given.

Additionally, parameters values are accessible through a dictionary returned by the instance method `getParamsValue`:

In [15]:
rkScheme.getParamsValue()

{'nPoints': 5,
 'ptsType': 'EQUID',
 'quadType': 'LOBATTO',
 'form': 'Z2N',
 'rkScheme': 'BE',
 'nStepsPerPoint': 1}

Also, the normalized time point coordinates for the block are available through the `points` attribute:

In [16]:
rkScheme.points

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

One can also change the parameters, provided that those are accepted by the parameter class. For instance, if we want to change the `rkScheme` parameter to `EI` (method that does not exists):

In [17]:
from blockops.utils.params import ParamError

try:
    rkScheme = BlockSchemeClass(5, rkScheme='EI')
except ParamError as err:
    print(f'ParamError : {err.msg}')
    print(f' -- name : {err.name}')
    print(f' -- value : {err.value}')
    print(f' -- reason : {err.reason}')

ParamError : rkScheme=EI -> is not in ['BE', 'FE', 'RK21', 'TRAP', 'RK2', 'GAUSS-LG', 'SDIRK2', 'RK3', 'RK53', 'SDIRK3', 'RK4', 'SDIRK54', 'RK65', 'EXACT'] (MultipleChoices)
 -- name : rkScheme
 -- value : EI
 -- reason : is not in ['BE', 'FE', 'RK21', 'TRAP', 'RK2', 'GAUSS-LG', 'SDIRK2', 'RK3', 'RK53', 'SDIRK3', 'RK4', 'SDIRK54', 'RK65', 'EXACT'] (MultipleChoices)


We see in the raised exception that the parameter value is not accepted, and indication are done on how to solve it.

Same goes for the `ptsType` parameter, for which we can provide either a string or a list of float values with predefined points. If the later are wrongly provided, we can see the reason why in the exception message :

In [18]:
try:
    rkScheme = BlockSchemeClass(5, ptsType=[-1, 0, 1])
except ParamError as err:
    print(f'ParamError : {err.msg}')
    print(f' -- name : {err.name}')
    print(f' -- value : {err.value}')
    print(f' -- reason : {err.reason}')

ParamError : ptsType=[-1, 0, 1] -> is not in ['EQUID', 'LEGENDRE', 'CHEBY-1', 'CHEBY-2', 'CHEBY-3', 'CHEBY-4', 'points are not included in [0, 1] (CustomPoints)'] (MultipleChoices)
 -- name : ptsType
 -- value : [-1, 0, 1]
 -- reason : is not in ['EQUID', 'LEGENDRE', 'CHEBY-1', 'CHEBY-2', 'CHEBY-3', 'CHEBY-4', 'points are not included in [0, 1] (CustomPoints)'] (MultipleChoices)
